In [2]:
#데이터 전처리 하기
import pandas as pd
import numpy as np
import xarray as xr
import dask.array as da
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator


In [5]:
#2005_wind파일
# .nc 파일 열기
dataset = xr.open_dataset('2005_wind.nc', engine='netcdf4')

# 필요한 변수 'v10'와 'u10' 추출
v10_data = dataset['v10']
u10_data = dataset['u10']


# 아래와 같이 수정하여 numpy datetime64 객체로 변환합니다.
start_time = v10_data['time'].min().values.astype('datetime64[ns]')
end_time = v10_data['time'].max().values.astype('datetime64[ns]')

# 이제 np.arange()를 사용하여 새로운 시간 배열을 생성합니다.
step = np.timedelta64(6, 'h')
new_time_np = np.arange(start_time, end_time, step)


new_time = da.from_array(new_time_np, chunks=1)  # Dask 배열로 변환

# Dask 배열을 사용하여 보간 수행
v10_interp = v10_data.interp(time=new_time_np, method='linear')
u10_interp = u10_data.interp(time=new_time_np, method='linear')


# 결과를 계산 및 저장
v10_interp = v10_interp.compute()
u10_interp = u10_interp.compute()

# 새로운 데이터셋 생성
new_dataset = xr.Dataset({'v10': v10_interp, 'u10': u10_interp})


# 2016_wind파일
dataset2 = xr.open_dataset('2006_wind.nc', engine='netcdf4')

# 필요한 변수 'v10'와 'u10' 추출
v10_data2 = dataset2['v10']
u10_data2 = dataset2['u10']

# 아래와 같이 수정하여 numpy datetime64 객체로 변환합니다.
start_time2 = v10_data2['time'].min().values.astype('datetime64[ns]')
end_time2 = v10_data2['time'].max().values.astype('datetime64[ns]')

# 이제 np.arange()를 사용하여 6시간 간격으로 새로운 시간 배열을 생성합니다.
new_time_np2 = np.arange(start_time2, end_time2, step)

# Dask 배열로 변환
new_time2 = da.from_array(new_time_np2, chunks=1)

# Dask 배열을 사용하여 보간 수행
v10_interp2 = v10_data2.interp(time=new_time_np2, method='linear')
u10_interp2 = u10_data2.interp(time=new_time_np2, method='linear')

# 결과를 계산 및 저장
v10_interp2 = v10_interp2.compute()
u10_interp2 = u10_interp2.compute()

# 새로운 데이터셋 생성
new_dataset2 = xr.Dataset({'v10': v10_interp2, 'u10': u10_interp2})

# 두 데이터셋을 연결하여 전체 연도 데이터 생성
final_dataset = xr.concat([new_dataset, new_dataset2], dim='time')

# 저장할 파일 이름 설정
output_filename = 'interpolated_wind.nc'

# 데이터셋을 .nc 파일로 저장
final_dataset.to_netcdf(output_filename)

# 파일을 닫습니다.
dataset.close() 
dataset2.close()